In [ ]:
#| default_exp callback

In [ ]:
#|export
import torch
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
from collections import defaultdict
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt

class ActionTrackingCallback(BaseCallback):
    def __init__(self, log_freq=100, verbose=0):
        super(ActionTrackingCallback, self).__init__(verbose)
        self.log_freq = log_freq
        self.episode_count = 0
        self.action_history = defaultdict(list)
        self.current_episode_actions = []
        self.max_generations = None
        self.step_count = 0
        self.writer = None

    def _on_training_start(self):
        self.writer = SummaryWriter(self.logger.dir)

    def _on_step(self) -> bool:
        self.step_count += 1
        info = self.locals['infos'][0]
        action = self.locals['actions'][0]
        current_generation = info['current_generation']
        
        if self.max_generations is None:
            self.max_generations = self.training_env.get_attr('max_generations')[0]

        # Normalize action from [-1, 1] to [0, 1]
#         normalized_action = (action.item() + 1) / 2
    # Use the normalized action from the environment
        normalized_action = info['normalized_action']

        self.current_episode_actions.append((current_generation, normalized_action))

        if self.locals['dones'][0]:
            self.episode_count += 1
            for gen, action in self.current_episode_actions:
                self.action_history[gen].append(action)
            self.current_episode_actions = []

            if self.episode_count % self.log_freq == 0:
                self._log_action_data()

        return True

    def _log_action_data(self):
        action_data = {}
        for gen in range(self.max_generations+1):
            if gen in self.action_history:
                avg_action = np.mean(self.action_history[gen])
                action_data[f"generation_{gen}"] = avg_action

        # Log all generation data at once
        self.writer.add_scalars("average_actions", action_data, self.step_count)

        # Clear the action history after logging
        self.action_history.clear()

    def on_training_end(self):
        # Final logging of action data
        self._log_action_data()
        if self.writer:
            self.writer.close()
            
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
from collections import defaultdict
class AverageFinalGenerationCallback(BaseCallback):
    def __init__(self, log_freq=100, verbose=0):
        super(AverageFinalGenerationCallback, self).__init__(verbose)
        self.log_freq = log_freq
        self.phenotypes = []
        self.genetic_variances = []
        self.episode_count = 0
        self.all_action_values = []  # Store all action values
        self.current_episode_actions = []

    def _on_step(self) -> bool:
        action = self.locals['actions'][0]
        self.current_episode_actions.append(action.item())
        
        for env_idx, done in enumerate(self.locals['dones']):
            if done:
                info = self.locals['infos'][env_idx]
                if 'final_generation' in info:
                    self.episode_count += 1
                    final_gen_info = info['final_generation']
                    
                    phenotype = final_gen_info['max_phenotype']
                    if isinstance(phenotype, torch.Tensor):
                        phenotype = phenotype.cpu().numpy()
                    self.phenotypes.append(phenotype)
                    
                    genetic_variance = final_gen_info['genetic_variance']
                    if isinstance(genetic_variance, torch.Tensor):
                        genetic_variance = genetic_variance.cpu().numpy()
                    self.genetic_variances.append(genetic_variance)
                    
                    # Store actions for this episode
                    self.all_action_values.extend(self.current_episode_actions)
                    self.current_episode_actions = []  # Reset for next episode

                    # Log every log_freq episodes
                    if self.episode_count % self.log_freq == 0:
                        self._log_data()

        return True

    def _log_data(self):
        avg_phenotype = np.mean(self.phenotypes)
        avg_genetic_variance = np.mean(self.genetic_variances)
        self.logger.record("final_generation/max_phenotype", avg_phenotype)
        self.logger.record("final_generation/genetic_variance", avg_genetic_variance)
        
        # Plot action values
        self._plot_action_values()
        
        # Reset lists for next logging period
        self.phenotypes = []
        self.genetic_variances = []
        self.all_action_values = []  # Store all action values

    def _plot_action_values(self):
        plt.figure(figsize=(12, 6))
        plt.plot(self.all_action_values)
        plt.title("Action Values Across All Training Steps")
        plt.xlabel("Training Step")
        plt.ylabel("Action Value")
        plt.ylim(0, 1)  # Assuming action values are between 0 and 1
#         plt.savefig(f"action_values_plot_{self.episode_count}.png")
        plt.close()

    def on_training_end(self):
        # Plot final action values chart
        self._plot_action_values()
import numpy as np
from stable_baselines3.common.callbacks import BaseCallback
from collections import defaultdict

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)